<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/echo/echo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This test is designed to hammer Cytoscape by sending a long sequence of short and long operations. On its own, it's an effective test of Jupyter-Bridge -- it verifies that a Colab Notebook can keep a reliable channel to a private Cytoscape or a collection of Cytoscape workstations.

But when this test is executed in multiple Colabs (each connected to its own Cytoscape in its own VM), it verifies that Jupyter-Bridge can process a high volume of requests while keeping good track of requests and replies.

The **Echo test** sends a long series of short messages to Cytoscape, and gets short replies. This stresses the Jupyter-Bridge message handling and thread allocation features.

The **Network Load test** sends a series of network import and table import messages. This stresses Cytoscape and shows the client's ability to wait a long time for a reply.

The **Column Load test** fetches a large amount of column data repeatedly. This stresses Jupyter-Bridge's ability to keep multiple large responses straight and moving to the clients. 

*Note* that when these tests are executed on multiple Colabs, each Colab should operate continuously without waiting (much) for activity on other Colabs to complete.


---
# Setup data files, py4cytoscape and Cytoscape connection
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

## Setup: Fetch latest py4cytoscape




**Note that you can fetch from the latest Github unreleased version by setting _PY4CYTOSCAPE to 'git+https://github.com/cytoscape/py4cytoscape' immediately before the exec() call. To fetch a particular branch, add '@' to the end (e.g., 'git+https://github.com/cytoscape/py4cytoscape@0.0.12').**

To load the default (PyPI) py4cytoscape version, do not set _PY4CYTOSCAPE at all.

In [ ]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.6.0' # optional
import requests

exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)

IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cytoscape/py4cytoscape (to revision 1.6.0) to /tmp/pip-req-build-ockgabaq


  Running command git clone -q https://github.com/cytoscape/py4cytoscape /tmp/pip-req-build-ockgabaq
  Running command git checkout -b 1.6.0 --track origin/1.6.0
  Switched to a new branch '1.6.0'
  Branch '1.6.0' set up to track remote branch '1.6.0' from 'origin'.


  Created wheel for py4cytoscape: filename=py4cytoscape-1.6.0-py3-none-any.whl size=170883 sha256=4f1bbbf7eeb134a18490ca5604eb00a3b70d4d597dea437da76797d3ea36f535
  Stored in directory: /tmp/pip-ephem-wheel-cache-yiwc1q4a/wheels/41/f8/55/7d9713a08400a67188976c9214b7e48e0da9cd32b53ad30109
  Created wheel for python-igraph: filename=python_igraph-0.9.11-py3-none-any.whl size=9074 sha256=998a69b4d1539efc0ecbd23605cdf5013ec7ce64d7f8403cdcf5f0183bb6bba6
  Stored in directory: /root/.cache/pip/wheels/7d/e4/0c/0c83f70bd0b99ce8aea47a21f8e52502169e7dd17808d12f30
Successfully built py4cytoscape python-igraph
NumExpr defaulting to 2 threads.
Loading Javascript client ... bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 on https://jupyter-bridge.cytoscape.org
ADVICE: WHEN RUNNING UNDER COLAB, DO NOT RE-RUN THIS CELL WITHOUT MANUALLY EXECUTING Runtime | Factory Reset Runtime FROM THE COLAB MENU FIRST.


<IPython.core.display.Javascript object>

## Setup: Sanity test to verify Cytoscape connection

By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [ ]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.9.1',
 'automationAPIVersion': '1.6.0',
 'py4cytoscapeVersion': '1.6.0',
 'jupyterBridgeVersion': '0.0.2'}

## Setup: Initialize a basic static message
The basic message is a random digit sequence that is unlikely to be used by any other Colab Notebook instance.

In [ ]:
import uuid as uu
my_uuid = str(p4c.get_browser_client_channel())

print(f'Fixed message: {my_uuid}')


Fixed message: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1


## Echo Test
Call Cytoscape's echo command with a unique message and get back the unique message.

The message consists of a random uuid (previously generated) combined with a monotonically increasing value.

Thus messages sent from this Colab instance are guaranteed to be unique in the universe, and each message sent from this Colab instance is guaranteed to be different from any previous messages.


In [ ]:
%%time

import py4cytoscape as p4c

for i in range(0, 1000):
  message = f'{i}: {my_uuid}'
  ret = p4c.command_echo(message)
  if message == ret[0]:
    print(f'Successful: {message}')
  else:
    print(f'Failure: got {ret}, expected {message}')


Successful: 0: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 1: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 2: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 3: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 4: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 5: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 6: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 7: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 8: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 9: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 10: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 11: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 12: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 13: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 14: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 15: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 16: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 17: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Successful: 18: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1
Suc

## Setup: Prepare Network Load and Column Tests

In [ ]:
res_mitab = p4c.sandbox_url_to("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
res_soft = p4c.sandbox_url_to("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")

def load_network():
  p4c.close_session(False)
  try:
    p4c.import_network_from_file("BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
  except:
    pass
  p4c.load_table_data_from_file("GDS112_full.soft", start_load_row=83, data_key_column_index=10, delimiters="\t")

## Network Load Test
Load a network and then load the node table data. Both are substantial files that were already staged on the Cytoscape sandbox in a previous step.

In [ ]:
%%time

for i in range(0, 10):
  load_network()
  print(f'Load network {i}: {my_uuid} successful')

In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 0: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 1: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


...

Load network 7: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 8: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 9: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
CPU times: user 3.59 s, sys: 516 ms, total: 4.11 s
Wall time: 11min 19s


## Setup: Get a Gold Copy of Node Attributes

In [ ]:
load_network()
all_columns = p4c.get_table_columns(table='node', columns=['shared name', 'name', 'psi-mi', 'ID_REF', 'IDENTIFIER', 'GSM1029', 'GSM1030', 'GSM1032', 'GSM1033', 'GSM1034', 'Gene title', 'Gene symbol', 'Platform_ORF', 'Chromosome annotation', 'GO:Function', 'GO:Process', 'GO:Component', 'GO:Function ID', 'GO:Process ID', 'GO:Component ID'])


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


## Column Load Test
Fetch all values for various columns and verify that they are successfully and consistently transferred.

In [ ]:
%%time

for i in range(0, 20):
  column_vals = p4c.get_table_columns(table='node', columns=['shared name', 'name', 'psi-mi', 'ID_REF', 'IDENTIFIER', 'GSM1029', 'GSM1030', 'GSM1032', 'GSM1033', 'GSM1034', 'Gene title', 'Gene symbol', 'Platform_ORF', 'Chromosome annotation', 'GO:Function', 'GO:Process', 'GO:Component', 'GO:Function ID', 'GO:Process ID', 'GO:Component ID'])
  print(f'Get columns {i}: {my_uuid} {"successful" if all_columns.equals(column_vals) else "failed"}')



Get columns 0: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 1: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 2: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 3: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 4: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 5: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 6: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 7: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 8: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 9: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 10: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 11: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 12: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 13: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 14: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21e1 successful
Get columns 15: bb1caeb4-e7eb-4007-9dbc-cb4ac4eb21